# dlt workshop | Homework Solution

## Question 1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:

```bash
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"
```

What's the version of dlt that you installed?

In [3]:
import dlt

In [4]:
dlt.__version__

'1.12.3'

### Answer: 1.12.3

## Question 2. dlt pipeline

Now let's create a pipeline. 

We need to define a destination for that. Let's use the `qdrant` one:

```python
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)
```

In this case, we tell dlt (and Qdrant) to create a folder with
our data, and the name for it will be `db.qdrant`

Let's run it:

```python
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)
```

How many rows were inserted into the `zoomcamp_data` collection?

Look for `"Normalized data for the following tables:"` in the trace output.

In [12]:
import requests

In [13]:
@dlt.resource(write_disposition="replace", name="llm_zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [9]:
from dlt.destinations import qdrant

In [14]:
qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [15]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

2025-07-03 23:45:32.640 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 23d57e02-b321-485b-a86c-702f48803960)') Falling back to other sources.
100%|██████████| 77.7M/77.7M [04:03<00:00, 320kiB/s] 
2025-07-04 00:05:36.853 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 34944a44-c001-4270-a5fd-f891f55a744f)') Falling back to other sources.
2025-07-04 00:22:40.862 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 131630f8-34dd-469c-82f2-42415a4a5

Run started at 2025-07-04 02:28:05.367645+00:00 and COMPLETED in 54 minutes and 37.39 seconds with 4 steps.
Step extract COMPLETED in 2.10 seconds.

Load package 1751597381.8741446 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.13 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- llm_zoomcamp_data: 948 row(s)

Load package 1751597381.8741446 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 32 minutes and 58.67 seconds.
Pipeline zoomcamp_pipeline load step completed in 32 minutes and 58.64 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /home/marcelonieva/Repos/llm_zoom/llm-zoomcamp/workshop-dlt/db.qdrant location to store data
Load package 1751597381.8741446 is LOADED and contains no failed jobs

Step run COMPLETED in 54 minutes and 37.38 seconds.
Pipeline zo

### Answer: 948 rows

## Question 3. Embeddings

When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the `meta.json` file created
in the target folder.

In [ ]:
import json

with open("db.qdrant/meta.json", "r") as f:
  print(json.dumps(json.load(f)['collections']['zoomcamp_tagged_data']['vectors'], indent=2))

{
  "fast-bge-small-en": {
    "size": 384,
    "distance": "Cosine",
    "hnsw_config": null,
    "quantization_config": null,
    "on_disk": null,
    "datatype": null,
    "multivector_config": null
  }
}


### Answer: fast-bge-small-en